## 참고: https://hleecaster.com/ml-linear-regression-example/

## 라이브러리 설치, 호출

!pip3 install -U scikit-learn<br>
!pip3 install pandas<br>
!pip3 install numpy<br>
!pip3 install matplotlib<br>
!pip3 install statsmodels<br>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Dense, Concatenate, InputLayer
from tensorflow.keras.optimizers import SGD, Adam

from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

from tensorflow.keras.models import Model

## 데이터 다운로드 (특별할인 판매)

In [2]:
import pandas as pd
TRAIN = pd.read_csv('LungCancer/train.csv')
VALID = pd.read_csv('LungCancer/valid.csv')

In [3]:
TRAIN.head()

,Relapse,CEA_Post,Lymphatic,Vascular,pStage,pT,pN,Perineural,Age
0,0,0.0,0,0,4,2,2,0,73
1,0,1.2,1,0,4,2,2,1,73
2,1,1.8,1,0,4,1,2,0,53
3,0,0.0,1,0,4,2,2,0,57
4,1,52.0,1,1,6,4,3,0,65


# Input, Feature 설정

In [5]:
TRAIN_y = TRAIN[['Relapse']]
TRAIN_x = TRAIN[ TRAIN.columns[1:] ]

VALID_y = VALID[['Relapse']]
VALID_x = VALID[ VALID.columns[1:] ]

In [6]:
TRAIN_x1 = TRAIN_x.iloc[:, :4]
TRAIN_x2 = TRAIN_x.iloc[:, 4:8]

In [7]:
VALID_x1 = VALID_x.iloc[:, :4]
VALID_x2 = VALID_x.iloc[:, 4:8]

In [9]:
TRAIN_x2

,pT,pN,Perineural,Age
0,2,2,0,73
1,2,2,1,73
2,1,2,0,53
3,2,2,0,57
4,4,3,0,65
...,...,...,...,...
387,3,2,0,57
388,4,1,0,63
389,2,3,0,63
390,1,2,0,57


## Keras, Multiple-input Logit 모델 architecture

In [10]:
# TRAIN_x1
InputA = Input(shape=(TRAIN_x1.shape[1],), dtype='int32', name='inputA')
x = Dense(10, activation='relu')(InputA)
OutputA = Dense(4, activation='relu')(x)

In [11]:
# TRAIN_x2
InputB = Input(shape=(TRAIN_x2.shape[1],), dtype='int32', name='inputB')
x = Dense(10, activation='relu')(InputB)
OutputB = Dense(4, activation='relu')(x)

In [12]:
Merged = Concatenate(axis=1)([OutputA, OutputB])
Output = Dense(1, activation='sigmoid')(Merged)

In [13]:
Model = tf.keras.models.Model(inputs=[InputA, InputB], outputs=Output)

In [14]:
Model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputA (InputLayer)            [(None, 4)]          0           []                               
                                                                                                  
 inputB (InputLayer)            [(None, 4)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 10)           50          ['inputA[0][0]']                 
                                                                                                  
 dense_2 (Dense)                (None, 10)           50          ['inputB[0][0]']                 
                                                                                              

In [15]:
Model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=0.001))

In [16]:
Model.fit(x=[TRAIN_x1, TRAIN_x2], y=TRAIN_y, epochs=300, shuffle=True)

Epoch 1/300
13/13 [==============================] - 2s 4ms/step - loss: 6.5751
Epoch 2/300
13/13 [==============================] - 0s 4ms/step - loss: 4.9823
Epoch 3/300
13/13 [==============================] - 0s 4ms/step - loss: 3.5897
Epoch 4/300
13/13 [==============================] - 0s 4ms/step - loss: 2.6114
Epoch 5/300
13/13 [==============================] - 0s 4ms/step - loss: 2.3076
Epoch 6/300
13/13 [==============================] - 0s 3ms/step - loss: 2.1647
Epoch 7/300
13/13 [==============================] - 0s 4ms/step - loss: 2.0441
Epoch 8/300
13/13 [==============================] - 0s 4ms/step - loss: 1.9445
Epoch 9/300
13/13 [==============================] - 0s 4ms/step - loss: 1.8436
Epoch 10/300
13/13 [==============================] - 0s 3ms/step - loss: 1.7559
Epoch 11/300
13/13 [==============================] - 0s 4ms/step - loss: 1.6599
Epoch 12/300
13/13 [==============================] - 0s 6ms/step - loss: 1.5707
Epoch 13/300
13/13 [=================

In [20]:
# pip install pydot

Note: you may need to restart the kernel to use updated packages.


In [21]:
tf.keras.utils.plot_model(model=Model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [18]:
# Call-back 함수
# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
CP = ModelCheckpoint(filepath='{epoch:03d}-{loss:.4f}-{val_loss:.4f}.hdf5',
            monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#minitor -> loss -> val_loss

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=4,
                       verbose=1, min_lr=1e-8)
# factor: Learning rate에 곱할 것.
#0.1 -> 0.08 -> 0.064 ....
#monitor='loss' -> monitor='val_loss'

CALLBACK = [CP, LR]

In [19]:
Model.fit(x=[TRAIN_x1, TRAIN_x2], y=TRAIN_y, epochs=5, shuffle=True,
          batch_size=32, callbacks=CALLBACK, validation_data = ([VALID_x1, VALID_x2], VALID_y))

Epoch 1/5
 1/13 [=>............................] - ETA: 0s - loss: 0.6333
Epoch 1: val_loss improved from inf to 0.59547, saving model to 001-0.6090-0.5955.hdf5
13/13 [==============================] - 1s 53ms/step - loss: 0.6090 - val_loss: 0.5955 - lr: 0.0010
Epoch 2/5
 1/13 [=>............................] - ETA: 0s - loss: 0.6260
Epoch 2: val_loss did not improve from 0.59547
13/13 [==============================] - 0s 9ms/step - loss: 0.6111 - val_loss: 0.5964 - lr: 0.0010
Epoch 3/5
 1/13 [=>............................] - ETA: 0s - loss: 0.6214
Epoch 3: val_loss did not improve from 0.59547
13/13 [==============================] - 0s 9ms/step - loss: 0.6082 - val_loss: 0.5956 - lr: 0.0010
Epoch 4/5
 1/13 [=>............................] - ETA: 0s - loss: 0.4471
Epoch 4: val_loss did not improve from 0.59547
13/13 [==============================] - 0s 9ms/step - loss: 0.6190 - val_loss: 0.6082 - lr: 0.0010
Epoch 5/5
 1/13 [=>............................] - ETA: 0s - loss: 0.6567
E